# **Indonesia Skincare Go-To-Market (GTM) Map Strategy**

---

This notebook processes Google Trends + Tokopedia + socio-economic indicators into a visualization-ready CSV for Looker Studio.

---

**Steps:**
1. Place all CSV files in the `data/raw/` folder (see filename list below).
2. Sort the cells from top to bottom.
3. Fetch the results in `data/processed/` and connect to Looker/Looker Studio (via Google Sheets/BigQuery/Upload file).

---

**Expected CSV (in `data/raw/`):**
- `geoMap_Province.csv` — column: `Province, Skincare_date`
- `timeline.csv` — columns: `Time, Skincare_Search_Interst`
- `Related_Topics.csv` — columns: `Related_Topics, Percentage`
- `Tokopedia_data.csv` — columns: `product_id, price, discount_percentage, sold_count, rating, review_count, total_stock, category, brand_name_guess, province, price_band`
- `Gini_ratio.csv` — columns: `Province, Gini Ratio ... 2024`
- `Population_by_Density.csv` — columns: `Province, Population Density by Province (people/km2)`
- `Population_by_Age.csv` — column: `Province, Number of Male/Female Population_... (age bins)`
- `Average_monthly_per_capita.csv` — column: `Province, ... 2011..2024`
- `Regional_Provincial_Wage.csv` — column: `Province, Regional/Provincial_Minimum_Wage_(Rupiah)_2020`

> **Note**: The data obtained is still below 2025 but is considered consistent with the situation in 2025.

---

## **Import Library**

---

In [ ]:
import os, pathlib
import re, math
from datetime import datetime
import numpy as np
import pandas as pd
from dateutil import parser as dtp
from glob import glob

---

## **Data Handling**

---

### **Model structure**

---

In [ ]:

# Setup folder structure
BASE_DIR = pathlib.Path('.').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROC_DIR = BASE_DIR / 'data' / 'processed'
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROC_DIR.mkdir(parents=True, exist_ok=True)
print('RAW_DIR :', RAW_DIR)
print('PROC_DIR:', PROC_DIR)


RAW_DIR : C:\Users\user\Documents\Coding\Portofolio\Indonesia_Skincare_GTM_Map\data\raw
PROC_DIR: C:\Users\user\Documents\Coding\Portofolio\Indonesia_Skincare_GTM_Map\data\processed


---

### **Normalize Province**

---

In [ ]:
# Normalize province names
PROVINCE_MAP = {
    # "DKI JAKARTA": "DKI Jakarta",
    # "DAERAH ISTIMEWA YOGYAKARTA": "DI Yogyakarta",
}

def norm_province(x: str) -> str:
    if pd.isna(x):
        return x
    s = str(x).strip()
    s = PROVINCE_MAP.get(s.upper(), s.title())
    return s

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r'\s+', '_', c.strip()).lower() for c in df.columns]
    return df

def to_date(x):
    if pd.isna(x):
        return pd.NaT
    try:
        return dtp.parse(str(x)).date()
    except Exception:
        return pd.NaT

def zscore(s):
    s = pd.Series(s, dtype=float)
    if s.std(ddof=0) == 0 or s.isna().all():
        return pd.Series([0]*len(s), index=s.index)
    return (s - s.mean())/s.std(ddof=0)

def minmax(s):
    s = pd.Series(s, dtype=float)
    vmin, vmax = s.min(), s.max()
    if pd.isna(vmin) or pd.isna(vmax) or vmin == vmax:
        return pd.Series([0.5]*len(s), index=s.index)
    return (s - vmin) / (vmax - vmin)

def polyfit_slope(y, x=None):
    if x is None:
        x = np.arange(len(y))
    try:
        coeff = np.polyfit(x, y, 1)
        return coeff[0]
    except Exception:
        return np.nan

def read_csv_safe(path, **kwargs):
    if not os.path.exists(path):
        print(f"[WARN] Missing file: {path}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(path, **kwargs)
    except Exception:
        df = pd.read_csv(path, encoding="latin-1", **kwargs)
    return df


***Insight:***

This script is an ETL/analytics toolkit that hardens the data pipeline for dependable business decisions: read_csv_safe robustly ingests CSVs (encoding fallback), clean_cols standardizes column names, norm_province harmonizes province keys for reliable joins, to_date parses dates, while minmax and zscore normalize metrics to comparable scales, and polyfit_slope summarizes trend momentum; together these utilities prevent broken joins, enable fair cross-province comparisons, and produce features needed to compute the opportunity score, price-to-nonfood, and sold per 100k ultimately driving province prioritization (Wave 1/Wave 2), pricing/assortment strategy (entry, mid-tier, premium-lite), stock/media allocation, and consistent ROI evaluation.

---

### **Geomap Handling**

---

In [ ]:
# Processing: Geomap (interest per province)
def process_geomap(path='data/raw/geoMap_Province.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df).rename(columns={df.columns[0]: "province"})
    df["province"] = df["province"].apply(norm_province)

    # take all numeric columns → average as interest_province
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if not num_cols:
        for c in df.columns[1:]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df["interest_province"] = df[num_cols].mean(axis=1, skipna=True) if num_cols else np.nan

    out = df[["province", "interest_province"]]
    out.to_csv('data/processed/trends_geomap_summary.csv', index=False)
    return out


***Insight:***

process_geomap builds a province-level demand signal from the Google Trends Geomap file. It safely reads geoMap_Province.csv, cleans columns and normalizes the province key, coerces all date columns to numeric, then computes interest_province as the mean of all numeric columns per province. It outputs data/processed/trends_geomap_summary.csv with province, interest_province. Business purpose: provide a join-ready demand metric for the dashboard (“Search Interest by Province”), feed the demand component of the opportunity score, and highlight demand–supply gaps to drive activation priorities, stock/media allocation, and rollout sequencing.

---

### **Related Topics Handling**

---

In [ ]:
# Processing: Related Topics
def process_related_topics(path='data/raw/Related_Topics.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df).rename(columns={"related_topics": "topic"})
    df["percentage_norm"] = minmax(df["percentage"])
    df.to_csv('data/processed/related_topics_clean.csv', index=False)
    return df

***Insight:***

process_related_topics prepares Google Trends related topics for downstream use in dashboards and planning. It safely reads Related_Topics.csv, cleans columns and renames related_topics → topic for consistency, and computes a 0–1 weight percentage_norm = minmax(percentage), preserving the ranking while making the score comparable across slices (note: Trends percentage is a relative index, not share; normalization keeps ordering). The output is saved to data/processed/related_topics_clean.csv. Business purpose: deliver content and paid-media pillars (TOF education, MOF problem–solution, BOF product), enable a province × topic matrix to localize activation, and prioritize creative/keywords expected to lift CTR, conversion, and ultimately sold per 100k. Best-practice add-ons: dedupe & canonicalize topics (synonyms/bilingual), tag stage (TOF/MOF/BOF) and valid-count, and if province/time is present consider recency-weighted scoring or top-N per province.

---

### **Tokopedia Handling**

---

In [ ]:
# Processing: Tokopedia aggregated
def process_tokopedia(path='data/raw/Tokopedia_data.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df, pd.DataFrame()
    df = clean_cols(df).rename(columns={"brand_name_guess": "brand"})
    if "province" in df.columns:
        df["province"] = df["province"].apply(norm_province)

    agg = df.groupby("province", dropna=False).agg(
        median_price=("price", "median"),
        avg_price=("price", "mean"),
        sold=("sold_count", "sum"),
        avg_rating=("rating", "mean"),
        review=("review_count", "sum"),
        stock=("total_stock", "sum"),
        n_listing=("product_id", "count")
    ).reset_index()

    # price_band distribution
    if "price_band" in df.columns:
        band = (df
                .assign(count=1)
                .pivot_table(index="province", columns="price_band", values="count", aggfunc="sum", fill_value=0))
        band.columns = [f"band_{str(c)}" for c in band.columns]
        agg = agg.merge(band.reset_index(), on="province", how="left")

    agg.to_csv('data/processed/tokopedia_agg_by_province.csv', index=False)

    weekly = agg.copy()
    weekly["week"] = pd.Timestamp.today().date().isoformat()
    weekly = weekly[["province", "week", "sold"]]

    return agg, weekly


***Insight:***

provided process_tokopedia as a core ETL block that converts raw Tokopedia listings into province-level supply & pricing signals for dashboards and scoring. It: ingests and cleans columns (renames brand_name_guess→brand, normalizes province), aggregates by province to compute median_price, avg_price, total sold (units), avg_rating, total review, total stock, and n_listing, builds a price-band distribution via pivot (creating band_* columns for counts per price tier), and exports to data/processed/tokopedia_agg_by_province.csv; it also emits a simple weekly snapshot (province, week, sold) for trend tracking. Business purpose: this feeds the Marketplace Summary, Median Price by Province, and Price Band Distribution views, and supplies inputs for key metrics like sold_per_100k = sold/population_total×100000 (penetration) and price_to_nonfood = median_price/avg_monthly_nonfood_2024 (affordability), which roll into the opportunity score and drive tactical choices (entry/value vs premium-lite, stock/media allocation, rollout sequencing).

---

### **Gini Handling**

---

In [ ]:
# Processing: Gini 
def process_gini(path='data/raw/Gini_ratio.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df)
    if "province" not in df.columns:
        df.rename(columns={df.columns[0]: "province"}, inplace=True)
    df["province"] = df["province"].apply(norm_province)

    target_cols = [c for c in df.columns if "urban_rural" in c and ("semester_2" in c or "september_2024" in c)]
    if not target_cols:
        target_cols = [c for c in df.columns if "2024" in c.lower()]
    latest = df[target_cols].mean(axis=1, skipna=True) if target_cols else np.nan
    out = df[["province"]].copy()
    out["gini_2024"] = latest
    out.to_csv('data/processed/gini_latest.csv', index=False)
    return out


***Insight:***

process_gini turns the Gini ratio into a join-ready provincial feature. It safely reads the CSV, cleans columns and normalizes province, selects the latest columns—preferring those containing urban_rural and semester_2/september_2024 (proxy for H2/Sept 2024 release); if missing, it falls back to any 2024 columns and computes an average as gini_2024, and (4) writes province, gini_2024 to data/processed/gini_latest.csv. Business purpose: provide a slow-moving inequality signal that informs assortment & pricing: high Gini ⇒ barbell assortment (value + premium simultaneously) and hyper-local targeting rather than province-wide averages. It can also act as a penalty/moderator in the Opportunity Score, improving rollout prioritization, media/stock allocation, and premium-lite vs value decisions.

---

### **Population Density Handling**

---

In [ ]:
# Processing: Population density
def process_density(path='data/raw/Population_by_Density.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df)
    if "province" not in df.columns:
        df.rename(columns={df.columns[0]: "province"}, inplace=True)
    df["province"] = df["province"].apply(norm_province)

    for c in df.columns:
        if "density" in c:
            den_col = c
            break
    else:
        den_col = df.columns[1]
    out = df[["province", den_col]].rename(columns={den_col: "density_2021"})
    out.to_csv('data/processed/density_2021.csv', index=False)
    return out

***Insight:***

process_density prepares the province-level population density as an “urban reach” signal for analytics and scoring. It: safely reads Population_by_Density.csv, cleans columns and normalizes the province key, locates the column containing “density” (or falls back to the second column), and outputs only province and density_2021 to data/processed/density_2021.csv. Business purpose: density proxies distribution and media reach efficiency (more audience per touchpoint, cheaper last-mile) and indicates brand/premium readiness—urban markets are more competitive but high-value. This feeds the province_master table, the Population Density visual, and the density_norm component in the opportunity score (e.g., weight +0.1), which informs rollout prioritization, stock/media allocation, and pricing/assortment strategy by province.

---

### **Population by Age Handling**

---

In [ ]:
# Processing: Population by Age
def process_pop_age(path='data/raw/Population_by_Age.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df, pd.DataFrame()
    df = clean_cols(df)
    if "province" not in df.columns:
        df.rename(columns={df.columns[0]: "province"}, inplace=True)
    df["province"] = df["province"].apply(norm_province)

    female_cols = [c for c in df.columns if c.startswith("number_of_female_population_")]
    male_cols = [c for c in df.columns if c.startswith("number_of_male_population_")]
    df["female_total"] = df[female_cols].sum(axis=1, skipna=True) if female_cols else np.nan
    df["male_total"] = df[male_cols].sum(axis=1, skipna=True) if male_cols else np.nan
    df["population_total"] = df["female_total"] + df["male_total"]

    def sum_age(prefix, bins):
        cols = [f"{prefix}{b}" for b in bins]
        cols = [c for c in cols if c in df.columns]
        return df[cols].sum(axis=1, skipna=True) if cols else np.nan

    bins_15_44 = ["15-19","20-24","25-29","30-34","35-39","40-44"]
    bins_20_39 = ["20-24","25-29","30-34","35-39"]

    df["female_15_44"] = sum_age("number_of_female_population_", bins_15_44)
    df["female_20_39"] = sum_age("number_of_female_population_", bins_20_39)

    out = df[["province","population_total","female_total","male_total","female_15_44","female_20_39"]]
    out.to_csv('data/processed/population_age_summary.csv', index=False)
    return out, df


***Insight:***

provided process_pop_age to turn raw demographic tables into an actionable TAM at the provincial level. It safely reads the CSV, standardizes columns and normalizes the province key for reliable joins, sums all female/male age columns into female_total and male_total and builds population_total, and robustly aggregates target age bands (only summing existing columns) to produce female_15_44 and female_20_39. The concise output is written to data/processed/population_age_summary.csv with province, population_total, female_total, male_total, female_15_44, female_20_39. Business purpose: provide denominators and targetable TAM for core metrics:

- Enable per-capita comparisons (e.g., sold_per_100k = sold / population_total × 100000)
- Construct the Demand Proxy (interest_province × female_20_39 / population_total) to prioritize provinces 
- Guide stock/media allocation by core segment size, and 
- Feed TAM visuals (absolute & per-100k) and bubble sizes in scatter charts.

Best-practice adds: compute female_20_39_per_100k and female_share_20_39 inside this function, enforce clean numeric types, align the demographic year with other datasets, and log provinces with incomplete age bands.

---

### **Average monthly per capita Handling**

---

In [ ]:
# Processing: Average monthly per capita 
def process_expenditure(path='data/raw/Average_monthly_per_capita.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df)
    if "province" not in df.columns:
        df.rename(columns={df.columns[0]: "province"}, inplace=True)
    df["province"] = df["province"].apply(norm_province)

    cands = [c for c in df.columns if c.endswith("_2024") or c.endswith("2024")]
    if not cands:
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        cands = [num_cols[-1]] if num_cols else []
    col_2024 = cands[0] if cands else df.columns[-1]
    out = df[["province", col_2024]].rename(columns={col_2024: "avg_monthly_nonfood_2024"})
    out.to_csv('data/processed/avg_monthly_nonfood_2024.csv', index=False)
    return out


***Insight:***

provided process_expenditure to build a purchasing-power indicator—the average monthly non-food expenditure per capita (2024) at the provincial level—which serves as the denominator for affordability. The function safely reads the CSV, standardizes columns and normalizes the province key for reliable joins, detects the 2024 column (suffix _2024/2024; if missing, falls back to the last numeric column), and outputs province, avg_monthly_nonfood_2024 to data/processed/avg_monthly_nonfood_2024.csv. Business purpose: this feeds price_to_nonfood = median_price / avg_monthly_nonfood_2024, which rolls into the Opportunity Score, guides entry vs premium-lite pricing, stock/media allocation, and rollout sequencing (Wave 1 for affordable markets; Wave 2 for value-uplift).

---

### **Regional/Provincial Wage Handling**

---

In [ ]:
# Processing: Regional/Provincial Wage
def process_wage(path='data/raw/Regional_Provincial_Wage.csv'):
    df = read_csv_safe(path)
    if df.empty:
        return df
    df = clean_cols(df)
    if "province" not in df.columns:
        df.rename(columns={df.columns[0]: "province"}, inplace=True)
    df["province"] = df["province"].apply(norm_province)

    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    col = num_cols[0] if num_cols else df.columns[1]
    out = df[["province", col]].rename(columns={col: "ump_2020"})
    out.to_csv('data/processed/ump_2020.csv', index=False)
    return out


***Insight:***

process_wage produces a macro purchasing-power indicator—the provincial minimum wage (UMP)—as a join-ready feature. It safely reads the CSV, cleans columns and normalizes the province key, picks the first numeric column as the wage value and labels it ump_2020, and exports province, ump_2020 to data/processed/ump_2020.csv. Business purpose: UMP serves as a baseline purchasing-power screen to guide entry vs step-up/premium-lite pricing, to validate price_to_nonfood insights, and to optionally moderate the Opportunity Score. Practically, it helps distinguish premium-ready markets from those needing trial/value offers.

---

### **Province Master, Demand vs Supply, Opportunity Score, Weekly Join Handling**

---

In [ ]:

# Builders: province_master, demand_vs_supply, opportunity_score, weekly join
def build_province_master(gini, density, pop_sum, expend, wage):
    out = None
    for f in [gini, density, pop_sum, expend, wage]:
        if f is None or f.empty:
            continue
        out = f if out is None else out.merge(f, on="province", how="outer")
    if out is not None and "population_total" in out.columns:
        out["female_share"] = out["female_total"] / out["population_total"]
    if out is not None:
        out.to_csv('data/processed/province_master.csv', index=False)
    return out

def demand_vs_supply(trends_geomap, tokopedia_agg, pop_summary):
    if trends_geomap is None or trends_geomap.empty or tokopedia_agg is None or tokopedia_agg.empty:
        return pd.DataFrame()
    df = trends_geomap.merge(tokopedia_agg, on="province", how="outer")
    if pop_summary is not None and not pop_summary.empty:
        df = df.merge(pop_summary[["province","population_total","female_20_39"]], on="province", how="left")
    df["female_20_39_share"] = df["female_20_39"] / df["population_total"]
    df["demand_proxy"] = df["interest_province"] * df["female_20_39_share"]
    df["sold_per_100k"] = (df["sold"] / df["population_total"]) * 1e5
    return df

def opportunity_score(demand_supply_df, province_master):
    if demand_supply_df is None or demand_supply_df.empty:
        return pd.DataFrame()
    df = demand_supply_df.copy()
    if province_master is not None and not province_master.empty:
        df = df.merge(province_master[["province","avg_monthly_nonfood_2024","ump_2020","gini_2024","density_2021"]], on="province", how="left")
    df["price_to_nonfood"] = df["median_price"] / df["avg_monthly_nonfood_2024"]

    demand_norm = minmax(df["demand_proxy"])
    supply_norm = minmax(df["sold_per_100k"])
    afford_pen = minmax(df["price_to_nonfood"])
    gini_pen = minmax(df["gini_2024"])
    density_norm = minmax(df["density_2021"])

    score = (0.5*demand_norm - 0.3*supply_norm - 0.1*afford_pen + 0.1*density_norm - 0.0*gini_pen)
    score = minmax(score) * 100.0
    df["opportunity_score"] = score.round(2)

    cols = ["province","interest_province","median_price","sold","sold_per_100k",
            "female_20_39","demand_proxy","avg_monthly_nonfood_2024","ump_2020",
            "gini_2024","density_2021","price_to_nonfood","opportunity_score"]
    cols = [c for c in cols if c in df.columns]
    out = df[cols].sort_values("opportunity_score", ascending=False)
    out.to_csv('data/processed/opportunity_score_by_province.csv', index=False)
    return out

def build_weekly_join(timeline_weekly, tokopedia_weekly):
    if timeline_weekly is None or timeline_weekly.empty or tokopedia_weekly is None or tokopedia_weekly.empty:
        return pd.DataFrame()
    df = tokopedia_weekly.merge(timeline_weekly[["week","interest"]], on="week", how="left")
    df.to_csv('data/processed/demand_vs_supply_weekly.csv', index=False)
    return df


***Insight:***

This “Builders” block is the backbone that fuses all features into decision-ready datasets for the dashboard and ROI tracking:

1. build_province_master(...): Outer-joins Gini, density, demographics, 2024 non-food spend, and UMP into a single province master; derives female_share. Output: province_master.csv.
Business link: one-stop, consistent lookup for affordability/TAM/reach/inequality—foundation for pricing, assortment, segmentation, and rollout.

2. demand_vs_supply(...): merges demand (interest_province) with marketplace supply and demographics; derives female_20_39_share, demand_proxy = interest * share, and sold_per_100k.
Business link: core table to spot demand–supply gaps vs mature markets—drives activation vs value-uplift plays.

3. opportunity_score(...): adds macros, computes price_to_nonfood, min–max normalizes features, then scores:
0.5*demand_norm - 0.3*supply_norm - 0.1*afford_pen + 0.1*density_norm - 0.0*gini_pen,
rescales to 0–100, outputs opportunity_score_by_province.csv.
Business link: a prioritization compass balancing potential (demand, density) vs constraints (high supply, affordability), guiding rollout sequencing, stock/media allocation, and entry/value vs premium-lite decisions.

4. build_weekly_join(...): joins weekly interest with weekly sold by week; outputs demand_vs_supply_weekly.csv.
Business link: track co-movement of interest and sales, measure post-activation Δ sold per 100k, and support evidence-based scaling.

---

### **Pipeline Running**

---

In [ ]:
# Run Pipeline
timeline_df, timeline_weekly = process_timeline()
geomap_df = process_geomap()
related_topics_df = process_related_topics()
tokopedia_agg, toko_weekly = process_tokopedia()

gini = process_gini()
density = process_density()
pop_sum, pop_full = process_pop_age()
expend = process_expenditure()
wage = process_wage()

prov_master = build_province_master(gini, density, pop_sum, expend, wage)
dsv = demand_vs_supply(geomap_df, tokopedia_agg, pop_sum)
opp = opportunity_score(dsv, prov_master)
weekly_join = build_weekly_join(timeline_weekly, toko_weekly)

# Print summary of output files
print("=== DONE ===")
print("Output CSV:")
for p in sorted(glob('data/processed/*.csv')):
    print(" -", p)


=== DONE ===
Output CSV:
 - data/processed\avg_monthly_nonfood_2024.csv
 - data/processed\density_2021.csv
 - data/processed\gini_latest.csv
 - data/processed\opportunity_score_by_province.csv
 - data/processed\population_age_summary.csv
 - data/processed\province_master.csv
 - data/processed\related_topics_clean.csv
 - data/processed\tokopedia_agg_by_province.csv
 - data/processed\trends_geomap_summary.csv
 - data/processed\trends_timeline_enhanced.csv
 - data/processed\ump_2020.csv


: 

***Insight:***

Data that has been successfully generated can be used for visualization.

---